In [1]:
# Import packages
import os
import time
import tensorflow as tf
from google.colab import drive

# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Import Remaining Packages (Post-Install)
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col, count


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkML").getOrCreate()
spark

In [4]:
# Connect to google drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# Read in the csv data from google drive
# Double check to make sure your pathway to the csv's is the same!!!!
spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/train_timeseries.csv')
train_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/train_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/test_timeseries.csv')
test_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/test_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/validation_timeseries.csv')
validation_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/validation_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/soil_data.csv')
soil_data_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/soil_data.csv'), sep=",", header=True, inferSchema=True)

In [6]:
# Preview soil data
soil_data_df.show()

+----+---------+----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+-------------------+-------------------+---+---+---+---+---+---+---+
|fips|      lat|       lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|         URB_LAND|         GRS_LAND|        FOR_LAND|      CULTRF_LAND|        CULTIR_LAND|          CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+----+---------+----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+-------------------+-------------------+---+---+---+---+---+---+---+
|1001|32.536382| -86.64449|       63|0.0419|0.2788|0.2984|

In [7]:
# Preview training time series data
train_timeseries_df.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|
+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+
|1001|2000-01-01|   0.22|100.51| 9.65|14.74| 13.51| 13.51|  20.96|  11.46|      9.5|14.65|  2.2|     2.94|     1.49|       1.46| 4.85|     6.04|     3.23|       2.81| null|
|1001|2000-01-02|    0.2|100.55|10.42|16.69| 14.71| 14.71|   22.8|  12.61|    10.18| 16.6| 2.52|     3.43|     1.83|        1.6| 5.33|     6.13|     3.72|       2.41| null|
|1001|2000-01-03|   3.65|100.15|11.76|18.49| 16.52| 16.52|  22.73|  15.32|     7.41|18.41| 4.03|     5.33|     2.66|       2.67| 7.53| 

In [8]:
from pyspark.sql import functions as F

# Increase scores by 1 to make room for 0 as 'No drought'
modified_train_timeseries_df = train_timeseries_df.withColumn('score', F.col('score') + 1)

# Fill null values with 0 to indicate 'No drought'
filled_train_timeseries_df = modified_train_timeseries_df.fillna({'score':0})

In [9]:
# Join training timeseries data with soil data
joined_df = filled_train_timeseries_df.join(soil_data_df, on='fips', how='left')
joined_df.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+----+----------+-------+------+-----+-----+------+------+--

In [11]:
# REVIEW DATAFRAME SHAPES
# Count of Unique Locations in Soil Data
soil_locations = soil_data_df.groupBy('fips').agg(count("*").alias("count"))
soil_locations.count()
print(f"Soil Data contains {soil_locations.count()} unique locations")

# Count of Unique Locations in Test Timeseries
timeseries_locations = train_timeseries_df.groupBy('fips').agg(count("*").alias("count"))
timeseries_locations.count()
print(f"Timeseries contains {timeseries_locations.count()} unique locations")
print("---")

# Print shapes of dataframes
print(f"Soil Data contains {soil_data_df.count()} rows and {len(soil_data_df.columns)} columns")
print(f"Timeseries contains {train_timeseries_df.count()} rows and {len(train_timeseries_df.columns)} columns")
print(f"Joined contains {joined_df.count()} rows and {len(joined_df.columns)} columns")

Soil Data contains 3109 unique locations
Timeseries contains 3108 unique locations
---
Soil Data contains 3109 rows and 32 columns
Timeseries contains 19300680 rows and 21 columns
Joined contains 19300680 rows and 52 columns


In [12]:
# Review datatypes
joined_df.printSchema()

root
 |-- fips: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- PRECTOT: double (nullable = true)
 |-- PS: double (nullable = true)
 |-- QV2M: double (nullable = true)
 |-- T2M: double (nullable = true)
 |-- T2MDEW: double (nullable = true)
 |-- T2MWET: double (nullable = true)
 |-- T2M_MAX: double (nullable = true)
 |-- T2M_MIN: double (nullable = true)
 |-- T2M_RANGE: double (nullable = true)
 |-- TS: double (nullable = true)
 |-- WS10M: double (nullable = true)
 |-- WS10M_MAX: double (nullable = true)
 |-- WS10M_MIN: double (nullable = true)
 |-- WS10M_RANGE: double (nullable = true)
 |-- WS50M: double (nullable = true)
 |-- WS50M_MAX: double (nullable = true)
 |-- WS50M_MIN: double (nullable = true)
 |-- WS50M_RANGE: double (nullable = true)
 |-- score: double (nullable = false)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- elevation: integer (nullable = true)
 |-- slope1: double (nullable = true)
 |-- slope2: double (nullable = true)
 

In [13]:
# Review score column
from pyspark.sql import functions as F
joined_df_scores = joined_df.groupBy('score').agg(count("*").alias("count")).orderBy(F.desc("count"))
joined_df_scores.show()

+------------------+--------+
|             score|   count|
+------------------+--------+
|               0.0|16543884|
|               1.0| 1480827|
|               2.0|  219135|
|               3.0|  123789|
|               4.0|   82801|
|               5.0|   45841|
|               6.0|   21806|
|            1.0001|    1037|
|            1.0002|     742|
|1.9998999999999998|     681|
|            1.0003|     642|
|            2.9999|     487|
|            1.0004|     486|
|            1.0005|     478|
|            1.0006|     459|
|2.0000999999999998|     455|
|            1.9998|     454|
|            1.0007|     409|
|            1.9996|     383|
|            1.9997|     353|
+------------------+--------+
only showing top 20 rows



In [14]:
# Review Min Max of data (primarily for confirming score column looks right)
summary_df = joined_df.summary("min", "max")
summary_df.show()

+-------+-----+-------+------+-----+------+------+------+-------+-------+---------+------+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+-----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+--------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+
|summary| fips|PRECTOT|    PS| QV2M|   T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|    TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|        lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+-------+-----+-------+------+-----+------+------+------+-------+-------+-

In [15]:
# Columns that we do not want to include as features
exclude_columns = ['score', 'lat', 'lon', 'fips', 'date']

# Set feature columns (excluding non-feature columns)
feature_columns = [col_name for col_name in joined_df.columns if col_name not in exclude_columns]
feature_columns

['PRECTOT',
 'PS',
 'QV2M',
 'T2M',
 'T2MDEW',
 'T2MWET',
 'T2M_MAX',
 'T2M_MIN',
 'T2M_RANGE',
 'TS',
 'WS10M',
 'WS10M_MAX',
 'WS10M_MIN',
 'WS10M_RANGE',
 'WS50M',
 'WS50M_MAX',
 'WS50M_MIN',
 'WS50M_RANGE',
 'elevation',
 'slope1',
 'slope2',
 'slope3',
 'slope4',
 'slope5',
 'slope6',
 'slope7',
 'slope8',
 'aspectN',
 'aspectE',
 'aspectS',
 'aspectW',
 'aspectUnknown',
 'WAT_LAND',
 'NVG_LAND',
 'URB_LAND',
 'GRS_LAND',
 'FOR_LAND',
 'CULTRF_LAND',
 'CULTIR_LAND',
 'CULT_LAND',
 'SQ1',
 'SQ2',
 'SQ3',
 'SQ4',
 'SQ5',
 'SQ6',
 'SQ7']

In [16]:
# Create VectorAssembler instance to combine feature columns into a single dense vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembler

VectorAssembler_aed609f6e868

In [17]:
# Apply VectorAssembler to create the dense vector column
df_with_features = assembler.transform(joined_df)
df_with_features.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            features|
+----+----------+-

In [18]:
# Apply feature scaling using StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
scaler

StandardScaler_8df94fa9d262

In [19]:
# Fit the StandardScaler to compute mean and standard deviation
scaler_model = scaler.fit(df_with_features)
scaler_model

StandardScalerModel: uid=StandardScaler_8df94fa9d262, numFeatures=47, withMean=true, withStd=true

In [20]:
# Transform the DataFrame to get scaled feature vectors
df_with_scaled_features = scaler_model.transform(df_with_features)
df_with_scaled_features.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+--------------------+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            feature

In [21]:
# Split the preprocessed data into a training and testing dataset
train_data, test_data = df_with_scaled_features.randomSplit([0.8, 0.2], seed=42)

In [29]:
from pyspark.ml.regression import RandomForestRegressor

# Define the model
rf = RandomForestRegressor(featuresCol='scaled_features', labelCol='score')

In [30]:
# Train the model
rf_model = rf.fit(df_with_scaled_features)
rf_model

RandomForestRegressionModel: uid=RandomForestRegressor_5c47a1f7eb49, numTrees=20, numFeatures=47

In [31]:
# Get feature importance
importances = rf_model.featureImportances
for i in range(len(feature_columns)):
    print(f"Importance of feature {feature_columns[i]}: {importances[i]}")

Importance of feature PRECTOT: 0.004526209802425842
Importance of feature PS: 0.12399749929309825
Importance of feature QV2M: 0.05479473685805943
Importance of feature T2M: 0.04086859461865931
Importance of feature T2MDEW: 0.025943110716290952
Importance of feature T2MWET: 0.023586927337032392
Importance of feature T2M_MAX: 0.17338335276760883
Importance of feature T2M_MIN: 0.014099692725788732
Importance of feature T2M_RANGE: 0.09298028096601159
Importance of feature TS: 0.0772115455677564
Importance of feature WS10M: 0.0011771157440440786
Importance of feature WS10M_MAX: 0.001249461808575479
Importance of feature WS10M_MIN: 0.0003496602561056845
Importance of feature WS10M_RANGE: 0.0012847162676935494
Importance of feature WS50M: 0.00021049019687031825
Importance of feature WS50M_MAX: 0.000589105442497059
Importance of feature WS50M_MIN: 0.00030529889545517205
Importance of feature WS50M_RANGE: 0.0009344726506569546
Importance of feature elevation: 0.09340885776929406
Importance of f

In [32]:
# PREPARE TEST DATA

# Increase scores by 1 to make room for 0 as 'No drought'
modified_test_timeseries_df = test_timeseries_df.withColumn('score', F.col('score') + 1)

# Fill null values with 0 to indicate 'No drought'
filled_test_timeseries_df = modified_test_timeseries_df.fillna({'score':0})

# Join training timeseries data with soil data
joined_test_df = filled_test_timeseries_df.join(soil_data_df, on='fips', how='left')

# Create VectorAssembler instance to combine feature columns into a single dense vector column
assembler_test = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Apply VectorAssembler to create the dense vector column
test_df_with_features = assembler_test.transform(joined_test_df)

# Apply feature scaling using StandardScaler
test_scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)

# Fit the StandardScaler to compute mean and standard deviation
test_scaler_model = test_scaler.fit(test_df_with_features)

# Transform the DataFrame to get scaled feature vectors
test_df_with_scaled_features = test_scaler_model.transform(test_df_with_features)
test_df_with_scaled_features.show()

+----+----------+-------+------+----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+--------------------+
|fips|      date|PRECTOT|    PS|QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            features|

In [33]:
from pyspark.ml.evaluation import RegressionEvaluator

# Make predictions
predictions = rf_model.transform(test_df_with_scaled_features)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="score", predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)
print("Mean Squared Error (MSE) on test data = %g" % mse)

evaluator = RegressionEvaluator(
    labelCol="score", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="score", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R Squared (R2) on test data = %g" % r2)

Root Mean Squared Error (RMSE) on test data = 0.590452
Mean Squared Error (MSE) on test data = 0.348633
Mean Absolute Error (MAE) on test data = 0.392064
R Squared (R2) on test data = -0.00549866
